# Capstone | Data Cleaning

## Small Business | Tax Office

* Student name: Miguel Santana
* Student pace: Full Time
* Scheduled project review date/time: 11/28/2020, 2 PM
* Instructor name: James Irving
* Blog post: [Medium](https://msantana269.medium.com/clustering-customer-churn-time-series-b51c9f59d691)

Follow the notebooks and analysis in the following order:

1) Data_Cleaning.ipynb

2) Clustering.ipynb

3) Customer_Churn.ipynb

## Abstract

Through data exploration and analysis of clientele data over a three-year period, recommendations for business development are provided below. The three sections of our analysis are provided in the following notebooks: Data_Cleaning.ipynb, Clustering.ipynb and Customer_Churn.ipynb. The most significant loss for the office occurs in clients between the ages of 25 and 35 coming from single income households. Our analysis outlines strategies for targeted marketing and customer retention. 

## Methodology
A business owner has tasked our team with analyzing their client base in order to understand customer retention as they run a seasonal tax office business. COVID-19 has severely compromised their ability to have face to face interactions with customers and this has been a long standing cornerstone of their business layout. Machine learning and deep learning algorithms will be used in order to process this request. Predictive models will offer the business owner key insight to be used in their end of year business meetings and planning as the 2021 tax season approaches. 

## Framework
The OSEMN framework will be used to navigate the project, models and interpretations culminating in actionable business recommendations. 

The OSEMN Framework

* Obtain  
* Scrub
* Explore 
* Model 
* Interpret

## Importing Packages

In [1]:
import os
import pandas as pd
import pandasql as ps
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('mode.chained_assignment', None)
plt.style.use('seaborn-poster') 
import warnings
warnings.filterwarnings('ignore')

# OBTAIN
### Importing 2016, 2017 & 2018 Tax Returns

In [2]:
df16 = pd.read_csv('client16.csv')
df17 = pd.read_csv('client17.csv')
df18 = pd.read_csv('client18.csv')

#### Focusing on electronic returns; each valid return (per year) should have some type of acknowledgment code (proof of transmission) in order to be considered in our analysis. 

In [3]:
# Dropping non efiled returns prior to merging
df16 = df16[df16['e-File ACK Date #1'].notna()]
df17 = df17[df17['e-File ACK Date #1'].notna()]
df18 = df18[df18['e-File ACK Date #1'].notna()]

### Merging Files

In [4]:
# Merging on most current file
df = pd.merge(df18,df16,how='outer',on='TP_ID')

In [5]:
# Merging on the updated file
df = pd.merge(df,df17,how='outer',on='TP_ID')

<div class="alert alert-block alert-info">

Unfortunately, the tax preparers entering data made several mistakes when entering client info and a merge on multiple columns leads to a significant loss in data. As a result, the column nulls will need to be filled with the respective columns of other years. After much trial an error, this has been discovered across almost every demographic column. The merge will begin with the most recent year, filling in only the older missing data.

</div>

# SCRUB
Each data frame carries the same column values and as such when they are joined the first table's columns are denoted with the addition of a '_x', the second table's columns are denoted with '_y' and the third table's columns carry the original column name. 

In order to fill the demographics, null values per '_x' column (2018) will be filled by corresponding '_y' columns (2017). The remaining null values will then be filled by corresponding columns with the original name (2016). This process will only be applied to the demographic values.

### Filing Demographics

In [6]:
fillcols=['FS','Age','Gender','Sp_Age','County','City','State','Zip',
          'Email','TP_Occupation','SP_Occupation']
for col in fillcols:
    df[f'{col}_x'].fillna(df[f'{col}_y'],inplace=True)
    df[f'{col}_x'].fillna(df[f'{col}'],inplace=True)

## Dropping Unneeded 'Fill' Columns

In [7]:
# loop here for drops
df = df.drop(['FS_y','FS','Age_y','Age','Gender_y','Gender','Sp_Age_y','Sp_Age',
              'City_y','City','State_y','State','County_y','County','Zip_y','Zip',
              'Email_y','Email','TP_Occupation_y','TP_Occupation','SP_Occupation_y',
              'SP_Occupation','Type_x','Type_y','Type'],axis=1)

## Renaming Merged Columns

In [8]:
df = df.rename(columns={'FS_x':'FamSize','TP_ID':'ID','Age_x':'Age','Gender_x':'Gender',
                        'Sp_Age_x':'Sp_Age','City_x':'City','State_x':'State',
                        'Zip_x':'Zip','County_x':'County','Email_x':'Email',
                        'TP_Occupation_x':'Occupation','SP_Occupation_x':'Household_Income',
                        'Preparer_x':'Preparer18','Ly_Fed_Tax_x':'FedTax18','AGI_x':'AGI18',
                        'Prep_Fee_x':'Fee18','Paid_x':'Paid18','ADMN-Prep1_x':'ADMIN18',
                        'e-File Product #1_x':'Efile18','e-File ACK Date #1_x':'EfileDate18',
                        'e-File ACK Code #1_x':'Code18','Preparer_y':'Preparer16',
                        'Ly_Fed_Tax_y':'FedTax16','AGI_y':'AGI16','Prep_Fee_y':'Fee16',
                        'Paid_y':'Paid16','ADMN-Prep1_y':'ADMIN16','e-File Product #1_y':'Efile16',
                        'e-File ACK Date #1_y':'EfileDate16','e-File ACK Code #1_y':'Code16',
                        'Preparer':'Preparer17','Ly_Fed_Tax':'FedTax17','AGI':'AGI17',
                        'Prep_Fee':'Fee17','Paid':'Paid17','ADMN-Prep1':'ADMIN17',
                        'e-File Product #1':'Efile17','e-File ACK Date #1':'EfileDate17',
                        'e-File ACK Code #1':'Code17'})

#### Rearrangement for Aesthetics

In [9]:
# Reorder years: 2016, 2017, 2018
df = df[['FamSize','ID','Gender','Age','Sp_Age','City','State','Zip','County',
         'Email','Occupation','Household_Income','Preparer16','FedTax16','AGI16',
         'Fee16','Paid16','Efile16','EfileDate16','Code16','Preparer17','FedTax17',
         'AGI17','Fee17','Paid17','Efile17','EfileDate17','Code17','Preparer18',
         'FedTax18','AGI18','Fee18','Paid18','Efile18','EfileDate18','Code18']]

## Null Values
#### Spouse Age

In [10]:
# Filling spouse ages 
df['Sp_Age'] = df['Sp_Age'].fillna(value=0)

#### Federal Tax Liability

In [11]:
# Many low income customers have a zero dollar federal tax liability (owed to government)
fedtax = ['FedTax16','FedTax17','FedTax18']
for col in fedtax:
    df[col] = df[col].fillna(value=0)

#### Preparer, AGI, Fee, Paid, Efile, EfileDate and Code (Per Year)

In [12]:
finacols = ['Preparer16','AGI16','Fee16','Paid16','Efile16','EfileDate16','Code16',
            'Preparer17','AGI17','Fee17','Paid17','Efile17','EfileDate17','Code17',
            'Preparer18','AGI18','Fee18','Paid18','Efile18','EfileDate18','Code18']
for col in finacols:
    df[col] = df[col].fillna(value='No Return')

#### Email, Occupation, Household Income

In [13]:
demcols = ['Email','Occupation','Household_Income']
for col in demcols:
    df[col] = df[col].fillna(value='Not Listed')

#### Family Size, Tax Payer Age and Zip Code

In [14]:
cols = ['FamSize','Age','Zip']
for col in cols:
    df = df[df[col].notna()]

### Verifying Null Counts

In [15]:
print('Missing Values : ', df.isnull().sum().values.sum()) # null values
print('\n')
print('Missing Values: \n', df.isna().sum()) # unique values per column

Missing Values :  0


Missing Values: 
 FamSize             0
ID                  0
Gender              0
Age                 0
Sp_Age              0
City                0
State               0
Zip                 0
County              0
Email               0
Occupation          0
Household_Income    0
Preparer16          0
FedTax16            0
AGI16               0
Fee16               0
Paid16              0
Efile16             0
EfileDate16         0
Code16              0
Preparer17          0
FedTax17            0
AGI17               0
Fee17               0
Paid17              0
Efile17             0
EfileDate17         0
Code17              0
Preparer18          0
FedTax18            0
AGI18               0
Fee18               0
Paid18              0
Efile18             0
EfileDate18         0
Code18              0
dtype: int64


In [16]:
# Uppercase 
uppercols = ['County','City','Occupation']
for col in uppercols:
    df[col] = df[col].str.upper()

### Preparer Codes

Per the owner:

* 1 = Maria
* 3 = Rudy
* 9 = Rose
* 10 = Lety
* 11 = Admin (secretary aiding completion)
* 12 = Esther
* 14 = Ana
* 17 = Admin (secretary aiding completion)
* 19 = Admin (secretary aiding completion)
* 21 = Admin (secretary aiding completion)
* 23 = Admin (secretary aiding completion)
* 25 = Admin (secretary aiding completion)
* 26 = Laura

Secretary names in the preparer field to be changed to admin are: Bianca, Mayra, None, Nallely, Angelica and Jesus.

In [17]:
# replacing preparer values with names
dictvals = {'1':'MARIA','3':'RUDY','9':'ROSE','10':'LETY','11':'ADMIN','12':'ESTHER',
            '14':'ANA','17':'ADMIN','19':'ADMIN','21':'ADMIN','23':'ADMIN',
            '25':'ADMIN','26':'LAURA','27':'ADMIN','28':'ADMIN','BIANCA':'ADMIN',
            'Mayra':'ADMIN','None':'ADMIN','Nallely':'ADMIN','Angelica':'ADMIN',
            'Jesus':'ADMIN','No Return':'No Return'}


prepcols = ['Preparer16','Preparer17','Preparer18']
for col in prepcols:
    df[col] = df[col].replace(dictvals)#.fillna()

## Segmentation Methodology

The metric that will be used in filtering is AGI (adjusted gross income) which represents a mandatory field (even if that field is zero) when sending a return to the Government. Null values represent clients that existed in one year but not another and are denoted by the fill value 'No Return.'

Three Years of Income

Sorting: customer lost, customer retained, return customer and new customer 

* 1) Customer Lost | 2016 - 2017 - (n/a)
* 2) Return Customer | 2016 - (n/a) - 2018
* 3) Customer Retained | 2016 - 2017 - 2018
* 4) Customer Retained | (n/a) - 2017 - 2018
* 5) Customer Lost | (n/a) - 2017 - (n/a)
* 6) New Customer | (n/a) - (n/a) - 2018

### Customers Lost

* 1) Customer Lost | 2016 - 2017 - (n/a)
* 5) Customer Lost | (n/a) - 2017 - (n/a)

In [18]:
# Keeping valid 2017 returns
lost = df[df['AGI17'] != 'No Return']

In [19]:
# Keeping valid 2017 returns with invalid 2018 values (customers lost)
lost = lost[lost['AGI18'] == 'No Return']

In [20]:
lost = lost[['FamSize','ID','Age','Gender','Sp_Age','City','State','Zip','County',
             'Email','Occupation','Household_Income','Efile16','EfileDate16','Paid16',
             'Efile17','EfileDate17','Paid17','Efile18','EfileDate18','Paid18',
             'Preparer17','FedTax17','AGI17','Fee17','Code17']]

In [21]:
lost['Paid'] = lost['Paid17']
lost['LastEfileDate'] = lost['EfileDate17']
lost['Customer_Churn'] = 'Yes'

In [22]:
# Last prep, tax, AGI, fee, paid and code for analysis
lost = lost.rename(columns={'Preparer17':'Preparer','FedTax17':'FedTax','AGI17':'AGI',
                            'Fee17':'Fee','Code17':'Code'})

### Customers Retained

* 3) Customer Retained | 2016 - 2017 - 2018
* 4) Customer Retained | (n/a) - 2017 - 2018

In [23]:
# Keeping valid 2017 returns
retained = df[df['AGI17'] != 'No Return']

In [24]:
# Keeping valid 2018 returns
retained = retained[retained['AGI18'] != 'No Return']

In [25]:
retained = retained[['FamSize','ID','Age','Gender','Sp_Age','City','State','Zip',
                     'County','Email','Occupation','Household_Income','Efile16',
                     'EfileDate16','Paid16','Efile17','EfileDate17','Paid17',
                     'Efile18','EfileDate18','Paid18','Preparer18','FedTax18',
                     'AGI18','Fee18','Code18']]

In [26]:
retained['Paid'] = retained['Paid18']
retained['LastEfileDate'] = retained['EfileDate18']
retained['Customer_Churn'] = 'No'

In [27]:
# Last prep, tax, AGI, fee, paid and code for analysis
retained = retained.rename(columns={'Preparer18':'Preparer','FedTax18':'FedTax',
                                    'AGI18':'AGI','Fee18':'Fee','Code18':'Code'})

### New Customers

* 6) New Customer | (n/a) - (n/a) - 2018

In [28]:
# Keeping invalid 2017 returns
new = df[df['AGI17'] == 'No Return']

In [29]:
# Keeping invalid 2016 returns
new = new[new['AGI16'] == 'No Return']

In [30]:
new = new[['FamSize','ID','Age','Gender','Sp_Age','City','State','Zip','County',
           'Email','Occupation','Household_Income','Efile16','EfileDate16','Paid16',
           'Efile17','EfileDate17','Paid17','Efile18','EfileDate18','Paid18','Preparer18',
           'FedTax18','AGI18','Fee18','Code18']]

In [31]:
new['Paid'] = new['Paid18']
new['LastEfileDate'] = new['EfileDate18']
new['Customer_Churn'] = 'New'

In [32]:
# Last prep, tax, AGI, fee, paid and code for analysis
new = new.rename(columns={'Preparer18':'Preparer','FedTax18':'FedTax','AGI18':'AGI',
                          'Fee18':'Fee','Code18':'Code'})

### Return Customers

* 2) Return Customer | 2016 - (n/a) - 2018

In [33]:
# Keeping invalid 2017 returns
returncust = df[df['AGI17'] == 'No Return']

In [34]:
# Keeping valid 2016 returns
returncust = returncust[returncust['AGI16'] != 'No Return']

In [35]:
# Keeping valid 2018 returns
returncust = returncust[returncust['AGI18'] != 'No Return']

In [36]:
returncust = returncust[['FamSize','ID','Age','Gender','Sp_Age','City','State',
                         'Zip','County','Email','Occupation','Household_Income',
                         'Efile16','EfileDate16','Paid16','Efile17','EfileDate17',
                         'Paid17','Efile18','EfileDate18','Paid18','Preparer18',
                         'FedTax18','AGI18','Fee18','Code18']]

In [37]:
returncust['Paid'] = returncust['Paid18']
returncust['LastEfileDate'] = returncust['EfileDate18']
returncust['Customer_Churn'] = 'return'

In [38]:
# Last prep, tax, AGI, fee, paid and code for analysis
returncust = returncust.rename(columns={'Preparer18':'Preparer','FedTax18':'FedTax',
                                        'AGI18':'AGI','Fee18':'Fee','Code18':'Code'})

# Updated Dataframe
#### Concatenating each updated section of our data frame representing customers that were new, lost, retained and returned.

In [39]:
frames = [lost, retained, new, returncust]
df2 = pd.concat(frames)

# Final Cleaning (Full DataFrame)

### Cleaning Time Based Values

In [40]:
cols = ['EfileDate16','EfileDate17','EfileDate18','LastEfileDate']
for col in cols:
    df2 = df2[df2[col] != 'Gads']
    df2 = df2[df2[col] != 'Yuma']
    df2 = df2[df2[col] != 'San']

### Cleaning Columns

In [41]:
# Cleaning invalid county value
df2 = df2[df2['County'] != 'UNITED STATES']

In [42]:
# Consolidating/converting house hold income values
df2['Household_Income'] = df2['Household_Income'].replace('Not Listed','UNEMPLOYED')

In [43]:
df2['Household_Income'] = df2['Household_Income'].map(lambda s :'Single Income' if s == 'UNEMPLOYED' else 'Dual Income')

### Converting Last Efile Date using Date Time

In [44]:
# adding a first value to create full date/extend dates in full format
df2['tempdate'] = '0' + df2['LastEfileDate']

In [45]:
# Last 4 values (year)
df2['Year'] = df2['tempdate'].str[-4:]
# Mid values -6 to -4 (day)
df2['Day'] = df2['tempdate'].str[-6:-4]

In [46]:
# Leftover data - should be in 2's and 3's
df2['temp'] = df2['tempdate'].str[0:-6]
# Pull the month (right values) but ID'd for convenience
df2['Month'] = df2['temp'].str[-2:]

In [47]:
# Full Date
df2['LastEfileDate'] = df2['Month']+'-'+df2['Day']+'-'+df2['Year']

In [48]:
df2.drop(['tempdate','Year','Day','temp','Month'],axis=1,inplace=True)

In [49]:
df2['LastDate'] = pd.to_datetime(df2['LastEfileDate'], format='%m-%d-%Y')
df2.set_index('LastDate', inplace=True)

In [50]:
# Feature engineer month name on last efile transmission per client
df2['Month'] = df2.index.strftime('%B')

In [51]:
# Feature engineer day of the week (name) on last efile transmission per client
df2['Day'] = df2.index.strftime('%A')

In [52]:
# Rearrange for aesthetics
df2 = df2[['FamSize','ID','Age','Gender','Sp_Age','City','State','Zip','County',
           'Email','Occupation','Household_Income','Efile16','EfileDate16','Paid16',
           'Efile17','EfileDate17','Paid17','Efile18','EfileDate18','Paid18','Preparer',
           'FedTax','AGI','Fee','Code','Paid','LastEfileDate','Month','Day','Customer_Churn']]

# Export File For Analysis

In [ ]:
df2.to_csv('taxdata.csv') # full data

# Notebooks
Please see our jupyter notebooks 'Clustering.ipynb' and 'Customer_Churn.ipynb' for additional analysis.